In [1]:
%load_ext autoreload
%autoreload 2
from manage_datasets import load_features_outcomes, load_features_labels, make_training_sets, make_stacked_sets, load_feature_list

device = 'cmod'
dataset_path = 'random_flattop_256_shots_60%_disruptive'

# Make training sets if they haven't been created yet
try:
    numeric_feats = load_feature_list(device, dataset_path)
except:
    make_training_sets(device, dataset_path, random_seed=0)
    numeric_feats = load_feature_list(device, dataset_path)
    

In [2]:
from auton_survival.preprocessing import Preprocessor

# Load and preprocess training, test, validation sets
features_train, outcomes_train = load_features_outcomes(device, dataset_path, 'train', numeric_feats)
features_test, outcomes_test = load_features_outcomes(device, dataset_path, 'test', numeric_feats)
features_val, outcomes_val = load_features_outcomes(device, dataset_path, 'val', numeric_feats)

# Create labels for binary classifier models according to the disruptive window
disruptive_window = 0.15 # seconds before disruption labeled as 'disruptive'
_, labels_train = load_features_labels(device, dataset_path, 'train', disruptive_window, numeric_feats)
_, labels_test = load_features_labels(device, dataset_path, 'test', disruptive_window, numeric_feats)
_, labels_val = load_features_labels(device, dataset_path, 'val', disruptive_window, numeric_feats)

# Fit the imputer and scaler to the training data and transform the training, test, and validation data
preprocessor = Preprocessor(cat_feat_strat='ignore', num_feat_strat='mean')
transformer=preprocessor.fit(features_train, cat_feats=[], num_feats=numeric_feats, one_hot=True, fill_value=-1)

x_train = transformer.transform(features_train)
x_test = transformer.transform(features_test)
x_val = transformer.transform(features_val)


In [3]:
from auton_survival.estimators import SurvivalModel
from auton_survival.metrics import survival_regression_metric
from run_models import get_train_times

y_train = outcomes_train
y_val = outcomes_val

times = get_train_times(y_train)

def evaluate_model(model:SurvivalModel):
    model.fit(x_train, y_train)
    predictions_val = model.predict_survival(x_val, times)
    metric_val = survival_regression_metric('ibs', y_val, predictions_val, times, y_train)
    return metric_val

In [4]:
import wandb
import numpy as np
import random

# Define sweep config
sweep_configuration = {
    'method': 'bayes',
    'name': 'cph-l2-sweep',
    'metric': {'goal': 'minimize', 'name': 'ibs'},
    'parameters': 
    {
        'l2': {'max': 0.01, 'min': 0.00001}
    }
}

# Initialize sweep by passing in config. 
# (Optional) Provide a name of the project.
sweep_id = wandb.sweep(sweep=sweep_configuration, project='notebook-sweep')

def main():
    run = wandb.init()

    l2 = wandb.config.l2

    model = SurvivalModel('cph', l2=l2)

    metric_val = evaluate_model(model)

    wandb.log({'ibs': metric_val})



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: ippacaqb
Sweep URL: https://wandb.ai/z-team/notebook-sweep/sweeps/ippacaqb


In [5]:
# Start sweep job.
wandb.agent(sweep_id, function=main)

wandb: Agent Starting Run: qajtdijb with config:
wandb: 	l2: 0.007894931011419106
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: zkeith (z-team). Use `wandb login --relogin` to force relogin


ibs,▁
ibs,0.14443


wandb: Agent Starting Run: z4u4g6qc with config:
wandb: 	l2: 0.0010663440544657057
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


ibs,▁
ibs,0.14542


wandb: Agent Starting Run: 30p8t7um with config:
wandb: 	l2: 0.007890805257087187
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

wandb: Agent Starting Run: un6t4bo8 with config:
wandb: 	l2: 0.006418223281466496
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


ibs,▁
ibs,0.14433


wandb: Agent Starting Run: gcf659lp with config:
wandb: 	l2: 0.0057081565217147615
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


ibs,▁
ibs,0.14429


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
